In [1]:
#%pip install llama-index nest_asyncio

In [1]:
from whisk.kitchenai_sdk.http_schema import (
    ChatCompletionRequest,
    ChatCompletionResponse,
    ChatCompletionChoice,
    ChatResponseMessage
)

In [10]:
from whisk.kitchenai_sdk.kitchenai import KitchenAIApp
from whisk.kitchenai_sdk.http_schema import (
    ChatCompletionRequest,
    ChatCompletionResponse,
    ChatCompletionChoice,
    ChatResponseMessage
)

# Initialize the app
kitchen = KitchenAIApp(namespace="whisk-jupyter-app-modified", version="0.0.2")

@kitchen.chat.handler("chat.completions.new")
async def handle_chat(request: ChatCompletionRequest) -> ChatCompletionResponse:
    """Simple chat handler that echoes back the last message"""



    return ChatCompletionResponse(
        model=request.model,
        choices=[
            ChatCompletionChoice(
                index=0,
                message=ChatResponseMessage(
                    role="assistant",
                    content=f"Echo demo: {request.messages[-1].content}"
                ),
                finish_reason="stop"
            )
        ]
    )

In [11]:
from whisk.kitchenai_sdk.schema import (
    ChatInput, 
    ChatResponse,
    DependencyType,
    SourceNode
)
# # Register dependency
# vector_store = MyVectorStore()
# kitchen.register_dependency(DependencyType.VECTOR_STORE, vector_store)

@kitchen.chat.handler("chat.rag", DependencyType.VECTOR_STORE, DependencyType.LLM)
async def rag_handler(chat: ChatInput, vector_store, llm) -> ChatResponse:
    """RAG-enabled chat handler"""
    # Get the user's question
    question = chat.messages[-1].content
    
    # Search for relevant documents
    retriever = vector_store.as_retriever(similarity_top_k=2)
    nodes = retriever.retrieve(question)
    
    # Create context from retrieved documents
    context = "\n".join(node.node.text for node in nodes)
    prompt = f"""Answer based on context: {context}\nQuestion: {question}"""
    
    # Get response from LLM
    response = await llm.acomplete(prompt)
    
    # Return response with sources
    return ChatResponse(
        content=response.text,
        sources=[
            SourceNode(
                text=node.node.text,
                metadata=node.node.metadata,
                score=node.score
            ) for node in nodes
        ]
    )

In [12]:
from whisk.kitchenai_sdk.schema import (
    WhiskStorageSchema,
    WhiskStorageResponseSchema
)
import time

@kitchen.storage.handler("storage")
async def storage_handler(data: WhiskStorageSchema) -> WhiskStorageResponseSchema:
    """Storage handler for document ingestion"""
    if data.action == "list":
        return WhiskStorageResponseSchema(
            id=int(time.time()),
            name="list",
            files=[]
        )
        
    if data.action == "upload":
        return WhiskStorageResponseSchema(
            id=int(time.time()),
            name=data.filename,
            label=data.model.split('/')[-1],
            metadata={
                "namespace": data.model.split('/')[0],
                "model": data.model
            },
            created_at=int(time.time())
        )

In [13]:
from whisk.config import WhiskConfig, ServerConfig
from whisk.router import WhiskRouter
import asyncio


nest_asyncio.apply()




# Configure and create router
config = WhiskConfig(server=ServerConfig(type="fastapi"))
router = WhiskRouter(kitchen_app=kitchen, config=config)

# Run the server

router.run(host="0.0.0.0", port=8000)




Task exception was never retrieved
future: <Task finished name='Task-28' coro=<Server.serve() done, defined at /home/epuerta/.local/share/hatch/env/virtual/kitchenai-whisk/jlwMXi0k/kitchenai-whisk/lib/python3.11/site-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/home/epuerta/.local/share/hatch/env/virtual/kitchenai-whisk/jlwMXi0k/kitchenai-whisk/lib/python3.11/site-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/home/epuerta/.local/share/hatch/env/virtual/kitchenai-whisk/jlwMXi0k/kitchenai-whisk/lib/python3.11/site-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/epuerta/.local/share/hatch/env/virtual/kitchenai-whisk/jlwMXi0k/kitchenai-whisk/lib/python3.11/site-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/

INFO:     127.0.0.1:59318 - "OPTIONS /v1/models HTTP/1.1" 200 OK
INFO:     127.0.0.1:59320 - "GET /v1/models HTTP/1.1" 200 OK
INFO:     127.0.0.1:47744 - "POST /v1/chat/completions HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2481637]
